<a href="https://colab.research.google.com/github/auliakharis/WSEEC-Student-Clustering/blob/main/WSEEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
 
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC, SVC
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
pd.options.display.precision = 15
 
import lightgbm as lgb
import xgboost as xgb
import time
import datetime
 
import json
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split
from sklearn.linear_model import Ridge, RidgeCV
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
 
import altair as alt
from IPython.display import HTML
from sklearn.linear_model import LinearRegression
 
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
 
import os
 
df = pd.read_csv("/content/sample_data/df_kluster.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  395 non-null    int64
 1   school      395 non-null    int64
 2   sex         395 non-null    int64
 3   age         395 non-null    int64
 4   address     395 non-null    int64
 5   famsize     395 non-null    int64
 6   Pstatus     395 non-null    int64
 7   Medu        395 non-null    int64
 8   Fedu        395 non-null    int64
 9   Mjob        395 non-null    int64
 10  Fjob        395 non-null    int64
 11  reason      395 non-null    int64
 12  guardian    395 non-null    int64
 13  traveltime  395 non-null    int64
 14  studytime   395 non-null    int64
 15  failures    395 non-null    int64
 16  schoolsup   395 non-null    int64
 17  famsup      395 non-null    int64
 18  paid        395 non-null    int64
 19  activities  395 non-null    int64
 20  nursery     395 non-null    int6

In [ ]:
df.isna().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [ ]:
categorical = ['school',\
               'sex',\
               'address',\
               'famsize',\
               'Pstatus',\
               'Mjob',\
               'Fjob',\
               'reason',\
               'guardian',\
               'schoolsup',\
               'famsup',\
               'paid',\
               'activities',\
               'nursery',\
               'higher',\
               'internet',\
               'romantic',
                ]

In [ ]:
#transform non-numerical labels to numerical labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df[categorical] = df[categorical].apply(lambda col: le.fit_transform(col))
df[categorical].head()

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,0,0,1,0,0,0,4,0,1,1,0,0,0,1,1,0,0
1,0,0,1,0,1,0,2,0,0,0,1,0,0,0,1,1,0
2,0,0,1,1,1,0,2,2,1,1,0,1,0,1,1,1,0
3,0,0,1,0,1,1,3,1,1,0,1,1,1,1,1,1,1
4,0,0,1,0,1,2,2,1,0,0,1,1,0,1,1,0,0


In [ ]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6
2,0,0,15,1,1,1,1,1,0,2,2,1,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10
3,0,0,15,1,0,1,4,2,1,3,1,1,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15
4,0,0,16,1,0,1,3,3,2,2,1,0,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10


In [ ]:
X = df.drop(['kluster'],axis=1)

In [ ]:
y = df['kluster']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver = 'lbfgs')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95        31
           1       1.00      0.95      0.97        39
           2       0.90      1.00      0.95         9

    accuracy                           0.96        79
   macro avg       0.95      0.97      0.96        79
weighted avg       0.96      0.96      0.96        79



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric = 'minkowski', p = 2, n_neighbors = 5)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83        31
           1       0.89      0.87      0.88        39
           2       0.86      0.67      0.75         9

    accuracy                           0.85        79
   macro avg       0.85      0.80      0.82        79
weighted avg       0.85      0.85      0.85        79



In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       1.00      1.00      1.00        24
           2       1.00      0.97      0.98        29

    accuracy                           0.99        79
   macro avg       0.99      0.99      0.99        79
weighted avg       0.99      0.99      0.99        79



In [ ]:
y_pred

array([1, 1, 1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 2, 2, 2,
       0, 2, 1, 0, 1, 0, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2, 2, 0, 0, 1, 1, 2,
       2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 0, 2, 2, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 2, 2, 1, 2], dtype=int32)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        31
           1       0.97      0.97      0.97        39
           2       0.69      1.00      0.82         9

    accuracy                           0.94        79
   macro avg       0.89      0.95      0.91        79
weighted avg       0.95      0.94      0.94        79



In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00         9

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79



In [ ]:
#INI OPSI KEDUA YA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import seaborn as sns

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state=42)

In [ ]:
kmeans.fit(df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [ ]:
labels = kmeans.fit_predict(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver = 'lbfgs')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       0.82      0.96      0.88        24
           2       0.96      0.79      0.87        29

    accuracy                           0.91        79
   macro avg       0.91      0.92      0.91        79
weighted avg       0.92      0.91      0.91        79



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric = 'minkowski', p = 2, n_neighbors = 5)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       0.96      1.00      0.98        24
           2       1.00      0.93      0.96        29

    accuracy                           0.97        79
   macro avg       0.97      0.98      0.98        79
weighted avg       0.98      0.97      0.97        79



In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       1.00      1.00      1.00        24
           2       1.00      0.97      0.98        29

    accuracy                           0.99        79
   macro avg       0.99      0.99      0.99        79
weighted avg       0.99      0.99      0.99        79



In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91        26
           1       1.00      0.79      0.88        24
           2       0.79      0.90      0.84        29

    accuracy                           0.87        79
   macro avg       0.89      0.87      0.88        79
weighted avg       0.89      0.87      0.87        79



In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        24
           2       1.00      1.00      1.00        29

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import seaborn as sns

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state=42)

In [ ]:
kmeans.fit(df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [ ]:
labels = kmeans.fit_predict(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver = 'lbfgs')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       0.81      0.88      0.84        24
           2       0.88      0.79      0.84        29

    accuracy                           0.89        79
   macro avg       0.89      0.89      0.89        79
weighted avg       0.89      0.89      0.89        79

